<a href="https://colab.research.google.com/github/umairnsr87/Hitachi-DataEngineer-HackerEarth/blob/master/data_engineer_hitachi_with%20baseline_models_and_improvements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [0]:
test = pd.read_csv('Test_1.csv')
train = pd.read_csv('Train_1.csv')


In [0]:
axx = pd.read_csv('Test_1.csv')
id_code = axx["id_code"]
del axx

In [0]:
train["mean_halt_times_source"].fillna(640.265896,inplace=True)
train["latitude_source"].fillna(50.859663,inplace=True)
train["longitude_source"]=train["longitude_source"].fillna(4.360846)
train["country_code_source"].fillna("whber",inplace=True)
train["country_code_destination"].fillna("whber",inplace=True)
train["mean_halt_times_destination"].fillna(164.41907514451,inplace=True)
train.dropna(axis=0,how='any',inplace=True)

In [285]:
train.isna().sum()

id_code                        0
current_date                   0
current_time                   0
source_name                    0
destination_name               0
train_name                     0
target                         0
country_code_source            0
longitude_source               0
latitude_source                0
mean_halt_times_source         0
country_code_destination       0
longitude_destination          0
latitude_destination           0
mean_halt_times_destination    0
current_year                   0
current_week                   0
current_day                    0
is_weekend                     0
dtype: int64

In [286]:
train.shape,test.shape

((1251, 19), (964, 18))

In [0]:
#removing the target
train.reset_index(inplace=True,drop=True)
target=train["target"]
#joining both train and test data for data cleaning
train["data"]="train"
train.drop(["target"],axis=1,inplace=True)
test["data"]="test"
data=pd.concat((train,test),ignore_index=True,axis=0)

In [288]:
#lets start to change the type of the data so that we can convert it into machine learning formal
data['source_name']=data['source_name'].str.replace("station","",case=False)
data['source_name']=data['source_name'].str.replace("$","")

#now change it from test and train
data['destination_name']=data['destination_name'].str.replace("station","",case=False)
data['destination_name']=data['destination_name'].str.replace("$","")

#now changing the datatype of these columns
data['source_name']=pd.to_numeric(data['source_name'])
data['destination_name']=pd.to_numeric(data['destination_name'])

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2215 entries, 0 to 2214
Data columns (total 19 columns):
id_code                        2215 non-null object
current_date                   2215 non-null object
current_time                   2215 non-null object
source_name                    2215 non-null int64
destination_name               2215 non-null int64
train_name                     2215 non-null object
country_code_source            2215 non-null object
longitude_source               2215 non-null float64
latitude_source                2215 non-null float64
mean_halt_times_source         2215 non-null float64
country_code_destination       2215 non-null object
longitude_destination          2215 non-null float64
latitude_destination           2215 non-null float64
mean_halt_times_destination    2215 non-null float64
current_year                   2215 non-null int64
current_week                   2215 non-null int64
current_day                    2215 non-null object
is_week

In [0]:
from sklearn.preprocessing import LabelEncoder
label=LabelEncoder()


In [0]:
#mapping the data on unique values
data["country_code_source"]=data["country_code_source"].map({'whber':1, 'qwnll':2,'wsluu':3,'logbp':4})
data["country_code_source"]=pd.to_numeric(data["country_code_source"])

In [291]:
data["country_code_destination"].unique()

array(['whber', 'aqfre', 'qwnll', 'zxdec', 'logbp', 'wsluu'], dtype=object)

In [0]:
#changing the data of destination
data["country_code_destination"]=data["country_code_destination"].map({'whber':1, 'qwnll':2,'wsluu':3,
                                                                  'logbp':4,'aqfre':5,'zxdec':6})
data["country_code_destination"]=pd.to_numeric(data["country_code_destination"])


In [0]:
data["current_day"]=data["current_day"].map({'Monday':1,'Tuesday':2,'Wednesday':3, 'Thursday':4, 'Friday':5, 
                         'Saturday':6, 'Sunday':7})
data["current_day"]=pd.to_numeric(data["current_day"])

In [294]:
data.tail(5)

,id_code,current_date,current_time,source_name,destination_name,train_name,country_code_source,longitude_source,latitude_source,mean_halt_times_source,country_code_destination,longitude_destination,latitude_destination,mean_halt_times_destination,current_year,current_week,current_day,is_weekend,data
2210,pnfrvyxsejnehwu,2016-10-29,09:14:45 AM,544,530,ICZVXA,1,3.710675,51.035896,309.014451,1,3.447848,51.092295,78.488439,2016,43,6,True,test
2211,omsilbnrgbvkeak,2016-10-29,10:17:59 AM,530,544,ICZVZA,1,3.447848,51.092295,78.488439,1,3.710675,51.035896,309.014451,2016,43,6,True,test
2212,vkjvqmaaguaeqde,2016-10-29,10:39:10 AM,178,147,ICRYYW,1,4.421101,51.217200,467.982659,1,4.356801,50.845658,634.164740,2016,43,6,True,test
2213,iutnjhogthfpymb,2016-10-29,10:59:55 AM,147,150,ICZVXY,1,4.356801,50.845658,634.164740,1,4.336531,50.835707,686.615607,2016,43,6,True,test
2214,xwqxedeqlnimclu,2016-10-29,11:48:37 AM,525,536,ICZVXW,1,3.216726,51.197226,164.419075,1,3.133864,51.312432,21.416185,2016,43,6,True,test


In [0]:
#feature engineering for date and time
#we are not doing more beacause there are several detailed columns are already present in the dataset
#data["date_and_time_combined"]=+" "+data["current_time"]
#data["current_date"]=pd.to_datetime(data["current_date"])


#getting the current month
xx=data["current_date"].str.split("-",expand=True)

data["current_month"]=xx[1]


In [296]:
data.isna().sum()

id_code                        0
current_date                   0
current_time                   0
source_name                    0
destination_name               0
train_name                     0
country_code_source            0
longitude_source               0
latitude_source                0
mean_halt_times_source         0
country_code_destination       0
longitude_destination          0
latitude_destination           0
mean_halt_times_destination    0
current_year                   0
current_week                   0
current_day                    0
is_weekend                     0
data                           0
current_month                  0
dtype: int64

In [0]:
#getting the hours of the time and converting it nto 24 hours format
a=data["current_time"].str.split(':',expand=True)
b=a[2].str.split(" ",expand=True)
a=a[0]
a=pd.to_numeric(a)
xx=np.where(b[1]=='AM',a+12,a)
xx=np.where(xx==24,0,xx)

hours=pd.DataFrame(xx)
data["hours_of_the_time"]=hours
del hours,xx
data.drop(["current_date","current_time",'id_code'],axis=1,inplace=True)

In [0]:
#changing the weekend into binary column
#data["is_weekend"].unique()
data["is_weekend"]=np.where(data["is_weekend"]==False,0,1)
data["is_weekend"]=pd.to_numeric(data["is_weekend"])

In [0]:
#making dummies for the train names for the trains which have values more that 5
#cat_cols=["train_name"]
#for col in cat_cols:
 #   freqs=data[col].value_counts()
  #  k=freqs.index[freqs>5][:-1]
   # for cat in k:
    #    name=col+'_'+cat
     #   data[name]=(data[col]==cat).astype(int)
    #del data[col]
    #print(col)



#we are deleting the train name for now 
# we will include the code when we will deal with the modelling process
del data["train_name"]

In [0]:
#converting the target into one hot encoding as it is a multiclass problem
#from sklearn.preprocessing import OneHotEncoder
#encoder = OneHotEncoder(handle_unknown='ignore')
#encoder.fit_transform(target)
#from sklearn.preprocessing import MultiLabelBinarizer
#MultiLabelBinarizer().fit_transform(target)


In [0]:
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
ss=StandardScaler()
ss=RobustScaler()
ms=MinMaxScaler()

In [301]:
ss

RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
             with_scaling=True)

In [302]:
#getting the training and testing data into the original form
train=data[data['data']=='train']
test=data[data['data']=='test']
print(train.shape,test.shape)
train.drop(['data'],axis=1,inplace=True)
test.drop(['data'],axis=1,inplace=True)
print(train.shape,test.shape)


(1251, 17) (964, 17)
(1251, 16) (964, 16)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
train=ss.fit_transform(train)
test=ss.fit_transform(test)

In [304]:
data.head()

,source_name,destination_name,country_code_source,longitude_source,latitude_source,mean_halt_times_source,country_code_destination,longitude_destination,latitude_destination,mean_halt_times_destination,current_year,current_week,current_day,is_weekend,data,current_month,hours_of_the_time
0,130,214,1,4.360846,50.859663,640.265896,1,4.482785,51.017648,306.523121,2016,30,3,0,train,07,11
1,248,248,1,5.497685,50.967057,39.476879,1,5.497685,50.967057,39.476879,2016,30,4,0,train,07,0
2,248,147,1,5.497685,50.967057,39.476879,1,4.356801,50.845658,634.164740,2016,30,4,0,train,07,0
3,177,130,1,4.482076,50.896456,171.020231,1,4.360846,50.859663,640.265896,2016,30,4,0,train,07,0
4,544,147,1,3.710675,51.035896,309.014451,1,4.356801,50.845658,634.164740,2016,30,4,0,train,07,20


In [305]:
train[:1]

array([[-0.224     ,  0.        ,  0.        ,  0.        , -0.29935883,
         1.33952936,  0.        ,  0.26907469,  0.62802152,  0.40548921,
         0.        , -4.        , -0.33333333,  0.        , -2.        ,
         0.13333333]])

In [0]:
#we are commentig this beacuse we have joined the (target)
#column because of eda to the datset

#train["target"]=target
#train["target"]=train["target"].map({'low':0,'high':2,'medium':1})



In [0]:
#sns.pairplot(train)

In [0]:
# hours_of_the time vs target 
#plt.figure(figsize=(10,8))
#sns.scatterplot(y=train["hours_of_the_time"],x=train["is_weekend"],hue=train["target"])
#plt.xlabel("weekends")
#plt.ylabel("hours in a day")
#plt.title("Hours in a day vs Weekoffs load ")
#as we can clearly see that on the working days the volumns is high at the time where people go to office andgo to home
#and on weekoffs the traffic is high in the morning and at afternoon
#there might be several reasons on the weekoff(Maybe the people are going to their native town or shopping)

In [0]:
#target vs destination name

#plt.figure(figsize=(9,6))
#sns.lineplot(y=train["target"], x=train['destination_name'])
#plt.xlabel("station location code")
#plt.ylabel("target in 0(low) ,1(medium),2(high)")
#plt.title("destination station name vs the volume")
#by this graph we can clearly see that as the stations are increasing the population volume is getting low
#this means that there are less number of people who atre commuting from very far places as compares to nearby places


In [0]:
#hours_of_the_time vs  mean_halt_source
#plt.figure(figsize=(6,6))
#sns.lineplot(x=train["hours_of_the_time"],y=train["mean_halt_times_source"])

#plt.xlabel("time in the day in 24 hours")
#plt.ylabel("Number of halt times")
#plt.title("Times the train halted on the day from 00:00-23:59:59")
#as we can see tthat the train halted nearly 13:00-17:00 the most
#it means the traffic would be the most at this time


In [0]:
#latitude source to latitude destination vs volume of the traffic
#plt.figure(figsize=(6,6))
#sns.lineplot(x=train["latitude_source"],y=train["latitude_destination"],hue=train["target"])
#we can clearly see that the major high volume between(50.25 to 51.25 latitude)
#we can clearly see that the normal volume between(49.75 to 51.50 latitude)
#we can clearly see that the normal volume between(49.75 to 51.50 latitude)




In [0]:
#latitude source to latitude destination vs volume of the traffic
#plt.figure(figsize=(6,6))
#sns.lineplot(x=train["longitude_source"],y=train["longitude_destination"],hue=train["target"])
#we can clearly see that the major high volume between(3 to 6 longitude)
#we can clearly see that the normal volume between(0 to 6 longitude)
#we can clearly see that the normal volume between(49.75 to 51.50 latitude)

In [0]:
#current_month vs the traffic vs week
#plt.figure(figsize=(6,6))
#sns.lineplot(train["mean_halt_times_source"],train["mean_halt_times_destination"],hue=train["target"])
#mean_halt times source and destination is alot
#this means there are alot of persons who prefer the train destination alot

In [0]:
#sns.kdeplot(train["current_month"])
# the frequesncy of september is more than in other months


In [0]:
#sns.lineplot(train["current_month"],train["mean_halt_times_source"])
#sns.lineplot(train["current_month"],train["mean_halt_times_destination"])
#plt.ylabel("mean halt time source and destination")

#we can clearly see that mean halt time is less both for source and destination in august(08) and maximum in july(07) 

In [0]:
#destination name vs target
#sns.lineplot(y=train["destination_name"],x=train["target"],hue=train["is_weekend"])

In [0]:
#train.drop(["current_year"],axis=1,inplace=True)
#test.drop(["current_year"],axis=1,inplace=True)

In [0]:
#function to get variable importances
#def plot_graph(a):
 #   plt.figure(figsize=(10,10))
  #  chart=sns.barplot(x=a["name_of_column"],y=a["feature_importance"])
  #  chart.set_xticklabels(chart.get_xticklabels(), rotation=45)
#def feature_importance_plot_rev(dataframe,classifier,limit_of_the_importance):
 #   x=list(zip(dataframe.columns,classifier.feature_importances_))
 #   xx=pd.DataFrame(x)
  #  xx.columns=["name_of_column","feature_importance"]
  #  xx=xx.sort_values("feature_importance",ascending=False)
  #  xx=xx[xx["feature_importance"]>limit_of_the_importance]
  #  plot_graph(xx)
  #  return xx

In [0]:
#training a model for getting the important features
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

#defining the classifier for feature importance
#rfc=RandomForestClassifier()
#dtc=DecisionTreeClassifier()

#saperating the train and test data
#X_train=train.drop(["target"],axis=1)
#Y_train=train["target"]

#fitting our model for variable importance
#rfc.fit(X_train,Y_train)
#dtc.fit(X_train,Y_train)

In [0]:
#getting the feature importance from random Forests classifier
#variable_importance_df=feature_importance_plot_rev(X_train,dtc,0)


In [0]:
#list1=list(variable_importance_df["name_of_column"])

In [0]:
#getting the feature importance from random Forests classifier
#feature_importance_plot_rev(X_train,rfc,0)
#commeting this beacuse the feture importance is same for both of them

In [0]:
#getting the most important function 
#df_important_features=pd.DataFrame()

#for i in list1:  
 # df_important_features=pd.concat((df_important_features,train[i]),axis=1)



In [0]:
#df_important_features_test=pd.DataFrame()
#for i in list1:  
 # df_important_features_test=pd.concat((df_important_features_test,test[i]),axis=1)


In [0]:
#df_important_features_test=ss.fit_transform(df_important_features_test).
#df_important_features=ss.fit_transform(df_important_features)

In [0]:
#resource expensive cell
#g = sns.PairGrid(df_important_features)
#g.map_upper(plt.scatter)
#.map_lower(sns.violinplot)
#g.map_diag(sns.kdeplot, lw=3, legend=False);


In [0]:
#now we will visualize whether we should go with th simple model or we should proceed with the complicated model
#temp=pd.concat((df_important_features,Y_train),axis=1)
#temp.head(5)
#from sklearn.decomposition import PCA
#pca=PCA(2)
#pca_fitted_transformed=pca.fit_transform(df_important_features)


In [0]:
#visualizing pca
#principalDf = pd.DataFrame(data = pca_fitted_transformed
 #            , columns = ['principal component 1', 'principal component 2'])
#finalDf = pd.concat([principalDf,Y_train], axis = 1)
#plt.figure(figsize=(8,8))
#sns.scatterplot(x=finalDf['principal component 1'], y=finalDf['principal component 2'],hue=finalDf['target'])
#plt.legend(["low","medium","high"])
#plt.show()

In [0]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score,GridSearchCV,RandomizedSearchCV,KFold,StratifiedKFold,train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier,GradientBoostingClassifier,AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [0]:

#lets go with the modelling process now
etc=ExtraTreesClassifier(class_weight='balanced')
dtc=DecisionTreeClassifier(class_weight='balanced')
rfc=RandomForestClassifier(class_weight='balanced')
gbc=GradientBoostingClassifier()
abc=AdaBoostClassifier(ExtraTreesClassifier(class_weight='balanced'))
lr=LogisticRegression(class_weight='balanced')

In [0]:
seed=77
#splittig the normal data into train and validation
train_x,val_x,train_y,val_y=train_test_split(train,Y_train,test_size=.30,random_state=seed)

In [324]:
#logistic classifier
lr.fit(train_x,train_y)
predicted=lr.predict(val_x)
predicted_proba=lr.predict_proba(val_x)
from sklearn.metrics import confusion_matrix,classification_report,r2_score,auc
#print(confusion_matrix(predicted,val_y))
#print(classification_report(predicted,val_y))

#print(r2_score(predicted,val_y))

#now training on the full dataset
lr.fit(train,Y_train)
predicted_final=lr.predict(test)

id_code=list(id_code)
predicted_final=list(predicted_final)
submit_lr=pd.DataFrame(list(zip(id_code,predicted_final)),columns=["id_code","target"])
submit_lr.target[submit_lr.target==0]='low'
submit_lr.target[submit_lr.target==1]='medium'
submit_lr.target[submit_lr.target==2]='high'
submit_lr.to_csv("submit_lr_stn.csv",index=False)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a

In [323]:
#adaboosting classifier
abc.fit(train_x,train_y)
predicted=abc.predict(val_x)
predicted_proba=abc.predict_proba(val_x)
from sklearn.metrics import confusion_matrix,classification_report,r2_score,auc
# print(confusion_matrix(predicted,val_y))
# print(classification_report(predicted,val_y))

# print(r2_score(predicted,val_y))
#now training on the full dataset
abc.fit(train,Y_train)
predicted_final=abc.predict(test)

id_code=list(id_code)
predicted_final=list(predicted_final)
submit_abc=pd.DataFrame(list(zip(id_code,predicted_final)),columns=["id_code","target"])
submit_abc.target[submit_abc.target==0]='low'
submit_abc.target[submit_abc.target==1]='medium'
submit_abc.target[submit_abc.target==2]='high'
submit_abc.to_csv("submit_abc_stn.csv",index=False)



/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

In [325]:
#gradientboosting classifier
gbc.fit(train_x,train_y)
predicted=gbc.predict(val_x)
predicted_proba=gbc.predict_proba(val_x)
#print(confusion_matrix(predicted,val_y))
#print(classification_report(predicted,val_y))

#print(r2_score(predicted,val_y))

#now training on the full dataset
gbc.fit(train,Y_train)
predicted_final=gbc.predict(test)

id_code=list(id_code)
predicted_final=list(predicted_final)
submit_gbc=pd.DataFrame(list(zip(id_code,predicted_final)),columns=["id_code","target"])
submit_gbc.target[submit_gbc.target==0]='low'
submit_gbc.target[submit_gbc.target==1]='medium'
submit_gbc.target[submit_gbc.target==2]='high'
submit_gbc.to_csv("submit_gbc_stn.csv",index=False)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [326]:
#extratrees classifier
etc.fit(train_x,train_y)
predicted=etc.predict(val_x)
predicted_proba=etc.predict_proba(val_x)
from sklearn.metrics import confusion_matrix,classification_report,r2_score,auc
#print(confusion_matrix(predicted,val_y))
#print(classification_report(predicted,val_y))

#print(r2_score(predicted,val_y))
#now training on the full dataset
etc.fit(train,Y_train)
predicted_final=etc.predict(test)

id_code=list(id_code)
predicted_final=list(predicted_final)
submit_etc=pd.DataFrame(list(zip(id_code,predicted_final)),columns=["id_code","target"])
submit_etc.target[submit_etc.target==0]='low'
submit_etc.target[submit_etc.target==1]='medium'
submit_etc.target[submit_etc.target==2]='high'
submit_etc.to_csv("submit_etc_stn.csv",index=False)



/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [327]:

#random forest classifier

rfc.fit(train_x,train_y)
predicted=rfc.predict(val_x)
predicted_proba=rfc.predict_proba(val_x)
#print(confusion_matrix(predicted,val_y))
#print(classification_report(predicted,val_y))

#print(r2_score(predicted,val_y))

#now training on the full dataset
rfc.fit(train,Y_train)
predicted_final=rfc.predict(test)

id_code=list(id_code)
predicted_final=list(predicted_final)
submit_rfc=pd.DataFrame(list(zip(id_code,predicted_final)),columns=["id_code","target"])
submit_rfc.target[submit_rfc.target==0]='low'
submit_rfc.target[submit_rfc.target==1]='medium'
submit_rfc.target[submit_rfc.target==2]='high'
submit_rfc.to_csv("submit_rfc_stn.csv",index=False)



/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


**USING THE MOST IMPORTANT PARAMETERS IN THE MODEL MAKING PROCESS.**

In [0]:

#lets go with the modelling process now
etc=ExtraTreesClassifier(class_weight='balanced')
dtc=DecisionTreeClassifier(class_weight='balanced')
rfc=RandomForestClassifier(class_weight='balanced')
gbc=GradientBoostingClassifier()
abc=AdaBoostClassifier(ExtraTreesClassifier(class_weight='balanced'))
lr=LogisticRegression(class_weight='balanced')

In [0]:
#SPLITTING THE MOST IMPORTNT FEATURES INTO TRAIN AND VAL
seed=77
#splittig the normal data into train and validation
train_x,val_x,train_y,val_y=train_test_split(df_important_features,Y_train,test_size=.30,random_state=seed)

In [71]:
#logistic classifier with most_important features
lr.fit(train_x,train_y)
predicted=lr.predict(val_x)
predicted_proba=lr.predict_proba(val_x)
from sklearn.metrics import confusion_matrix,classification_report,r2_score,auc
print(confusion_matrix(predicted,val_y))
print(classification_report(predicted,val_y))

print(r2_score(predicted,val_y))

#now training on the full dataset
lr.fit(df_important_features,Y_train)
predicted_final=lr.predict(df_important_features_test)

id_code=list(id_code)
predicted_final=list(predicted_final)
submit_lr_MI=pd.DataFrame(list(zip(id_code,predicted_final)),columns=["id_code","target"])
submit_lr_MI.target[submit_lr_MI.target==0]='low'
submit_lr_MI.target[submit_lr_MI.target==1]='medium'
submit_lr_MI.target[submit_lr_MI.target==2]='high'
submit_lr_MI.to_csv("submit_lr_MI.csv",index=False)


[[100  40  22]
 [ 23  23  26]
 [ 42  34  66]]
              precision    recall  f1-score   support

           0       0.61      0.62      0.61       162
           1       0.24      0.32      0.27        72
           2       0.58      0.46      0.52       142

    accuracy                           0.50       376
   macro avg       0.47      0.47      0.47       376
weighted avg       0.53      0.50      0.51       376

-0.251088636044388


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a

In [72]:
#adaboosting classifier
abc.fit(train_x,train_y)
predicted=abc.predict(val_x)
predicted_proba=abc.predict_proba(val_x)
from sklearn.metrics import confusion_matrix,classification_report,r2_score,auc
print(confusion_matrix(predicted,val_y))
print(classification_report(predicted,val_y))

print(r2_score(predicted,val_y))
#now training on the full dataset
abc.fit(df_important_features,Y_train)
predicted_final=abc.predict(df_important_features_test)

id_code=list(id_code)
predicted_final=list(predicted_final)
submit_abc_MI=pd.DataFrame(list(zip(id_code,predicted_final)),columns=["id_code","target"])
submit_abc_MI.target[submit_abc_MI.target==0]='low'
submit_abc_MI.target[submit_abc_MI.target==1]='medium'
submit_abc_MI.target[submit_abc_MI.target==2]='high'
submit_abc_MI.to_csv("submit_abc_MI.csv",index=False)



/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

[[121  43  26]
 [ 23  25  37]
 [ 21  29  51]]
              precision    recall  f1-score   support

           0       0.73      0.64      0.68       190
           1       0.26      0.29      0.27        85
           2       0.45      0.50      0.47       101

    accuracy                           0.52       376
   macro avg       0.48      0.48      0.48       376
weighted avg       0.55      0.52      0.53       376

-0.18547711709936432


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

In [76]:
#gradientboosting classifier
gbc.fit(train_x,train_y)
predicted=gbc.predict(val_x)
predicted_proba=gbc.predict_proba(val_x)
print(confusion_matrix(predicted,val_y))
print(classification_report(predicted,val_y))

print(r2_score(predicted,val_y))

#now training on the full dataset
gbc.fit(df_important_features,Y_train)
predicted_final=gbc.predict(df_important_features_test)

id_code=list(id_code)
predicted_final=list(predicted_final)
submit_gbc_MI=pd.DataFrame(list(zip(id_code,predicted_final)),columns=["id_code","target"])
submit_gbc_MI.target[submit_gbc_MI.target==0]='low'
submit_gbc_MI.target[submit_gbc_MI.target==1]='medium'
submit_gbc_MI.target[submit_gbc_MI.target==2]='high'
submit_gbc_MI.to_csv("submit_gbc_MI.csv",index=False)


[[113  43  36]
 [ 25  27  26]
 [ 27  27  52]]
              precision    recall  f1-score   support

           0       0.68      0.59      0.63       192
           1       0.28      0.35      0.31        78
           2       0.46      0.49      0.47       106

    accuracy                           0.51       376
   macro avg       0.47      0.48      0.47       376
weighted avg       0.54      0.51      0.52       376

-0.34013683446087994


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [77]:
#extratrees classifier
etc.fit(train_x,train_y)
predicted=etc.predict(val_x)
predicted_proba=etc.predict_proba(val_x)
from sklearn.metrics import confusion_matrix,classification_report,r2_score,auc
print(confusion_matrix(predicted,val_y))
print(classification_report(predicted,val_y))

print(r2_score(predicted,val_y))
#now training on the full dataset
etc.fit(df_important_features,Y_train)
predicted_final=etc.predict(df_important_features_test)

id_code=list(id_code)
predicted_final=list(predicted_final)
submit_etc_MI=pd.DataFrame(list(zip(id_code,predicted_final)),columns=["id_code","target"])
submit_etc_MI.target[submit_etc_MI.target==0]='low'
submit_etc_MI.target[submit_etc_MI.target==1]='medium'
submit_etc_MI.target[submit_etc_MI.target==2]='high'
submit_etc_MI.to_csv("submit_etc_MI.csv",index=False)



[[116  38  35]
 [ 27  32  29]
 [ 22  27  50]]
              precision    recall  f1-score   support

           0       0.70      0.61      0.66       189
           1       0.33      0.36      0.35        88
           2       0.44      0.51      0.47        99

    accuracy                           0.53       376
   macro avg       0.49      0.49      0.49       376
weighted avg       0.55      0.53      0.53       376

-0.3097776180780132


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [78]:

#random forest classifier

rfc.fit(train_x,train_y)
predicted=rfc.predict(val_x)
predicted_proba=rfc.predict_proba(val_x)
print(confusion_matrix(predicted,val_y))
print(classification_report(predicted,val_y))

print(r2_score(predicted,val_y))

#now training on the full dataset
rfc.fit(df_important_features,Y_train)
predicted_final=rfc.predict(df_important_features_test)

id_code=list(id_code)
predicted_final=list(predicted_final)
submit_rfc_MI=pd.DataFrame(list(zip(id_code,predicted_final)),columns=["id_code","target"])
submit_rfc_MI.target[submit_rfc_MI.target==0]='low'
submit_rfc_MI.target[submit_rfc_MI.target==1]='medium'
submit_rfc_MI.target[submit_rfc_MI.target==2]='high'
submit_rfc_MI.to_csv("submit_rfc_MI.csv",index=False)



[[108  47  32]
 [ 29  30  35]
 [ 28  20  47]]
              precision    recall  f1-score   support

           0       0.65      0.58      0.61       187
           1       0.31      0.32      0.31        94
           2       0.41      0.49      0.45        95

    accuracy                           0.49       376
   macro avg       0.46      0.46      0.46       376
weighted avg       0.51      0.49      0.50       376

-0.4297310593637258


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [0]:
#kfold=StratifiedKFold(n_splits=10,shuffle=True)

In [0]:

#classifiers = []
#classifiers.append(SVC(random_state=seed))
#classifiers.append(DecisionTreeClassifier(random_state=seed))
#classifiers.append(AdaBoostClassifier(DecisionTreeClassifier(random_state=state),
 #                                     random_state=seed,learning_rate=0.1))
#classifiers.append(RandomForestClassifier(random_state=seed))
#classifiers.append(ExtraTreesClassifier(random_state=seed))
#classifiers.append(GradientBoostingClassifier(random_state=seed))
#classifiers.append(MLPClassifier(random_state=seed))
#classifiers.append(KNeighborsClassifier())
#classifiers.append(LogisticRegression(random_state = seed))
#classifiers.append(LinearDiscriminantAnalysis())

#cv_results = []
#for classifier in classifiers :
 #   cv_results.append(cross_val_score(classifier, X_train, y = Y_train, scoring = "accuracy", cv = kfold, n_jobs=4))

#cv_means = []
#cv_std = []
#for cv_result in cv_results:
 #   cv_means.append(cv_result.mean())
  #  cv_std.append(cv_result.std())
#
#cv_res = pd.DataFrame({"CrossValMeans":cv_means,"CrossValerrors": cv_std,"Algorithm":["SVC","DecisionTree","AdaBoost",
#"RandomForest","ExtraTrees","GradientBoosting","MultipleLayerPerceptron","KNeighboors","LogisticRegression","LinearDiscriminantAnalysis"]})
